In [1]:
import pandas as pd
import numpy as np
from astropy.time import Time
from math import radians, cos, sin, asin, sqrt

In [42]:
csvfile = "../../Data/Location/ISS_FERMI_Light-1_Locations.csv"

data_pd = pd.read_csv(csvfile)
headers = list(data_pd.keys())
data    = [i for i in data_pd.to_numpy().T]
data    = dict(zip(headers,data))

In [23]:
headers
# data

['Time(ModJDate)',
 'Lat(deg)Light-1',
 'Lon(deg)Light-1',
 'Lon2(deg)Light-1',
 'Alt(km)Light-1',
 'Lat(deg)ISS',
 'Lon(deg)ISS',
 'Lon2(deg)ISS',
 'Alt(km)ISS',
 'Lat(deg)FERMI',
 'Lon(deg)FERMI',
 'Lon2(deg)FERMI',
 'Alt(km)FERMI']

In [24]:
def haversine(lon1, lat1, lon2, lat2):  #Calculate the great circle distance between two points on the earth 
                                        #(specified in decimal degrees)
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2]) # convert decimal degrees to radians 
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles.
    return c * r

In [52]:
distance_ISS_Light1 = [haversine(data['Lon(deg)Light-1'][i],data['Lat(deg)Light-1'][i],data['Lon(deg)ISS'][i],data['Lat(deg)ISS'][i]) for i in range(len(data['Time(ModJDate)']))]
distance_FERMI_Light1 = [haversine(data['Lon(deg)Light-1'][i],data['Lat(deg)Light-1'][i],data['Lon(deg)FERMI'][i],data['Lat(deg)FERMI'][i]) for i in range(len(data['Time(ModJDate)']))]

In [68]:
THRESHOLD   = 1000
ISS_idx_cut     = np.where([i < THRESHOLD for i in distance_ISS_Light1])[0]
ISS_time_cut    = data['Time(ModJDate)'][ISS_idx_cut]
# ISS_time_cut

array([59689.19444, 59689.19792, 59689.20139, ..., 59953.03125,
       59953.03472, 59953.06597])

In [103]:
ISS_time = [Time(i,format="mjd") for i in ISS_time_cut]
ISS_time = [(i.mjd,i.isot) for i in ISS_time]
ISS_time

[(59689.19444, '2022-04-20T04:39:59.616'),
 (59689.19792, '2022-04-20T04:45:00.288'),
 (59689.20139, '2022-04-20T04:50:00.096'),
 (59689.20486, '2022-04-20T04:54:59.904'),
 (59689.20833, '2022-04-20T04:59:59.712'),
 (59689.21181, '2022-04-20T05:05:00.384'),
 (59689.21528, '2022-04-20T05:10:00.192'),
 (59689.21875, '2022-04-20T05:15:00.000'),
 (59689.22222, '2022-04-20T05:19:59.808'),
 (59689.22569, '2022-04-20T05:24:59.616'),
 (59689.22917, '2022-04-20T05:30:00.288'),
 (59689.23264, '2022-04-20T05:35:00.096'),
 (59689.23611, '2022-04-20T05:39:59.904'),
 (59689.23958, '2022-04-20T05:44:59.712'),
 (59689.24306, '2022-04-20T05:50:00.384'),
 (59689.24653, '2022-04-20T05:55:00.192'),
 (59689.25, '2022-04-20T06:00:00.000'),
 (59689.25347, '2022-04-20T06:04:59.808'),
 (59689.25694, '2022-04-20T06:09:59.616'),
 (59689.26042, '2022-04-20T06:15:00.288'),
 (59689.26389, '2022-04-20T06:20:00.096'),
 (59689.26736, '2022-04-20T06:24:59.904'),
 (59689.27083, '2022-04-20T06:29:59.712'),
 (59689.27431,

In [108]:
# 300s is 5 minutes.. so 1000s for safety
TIME_THRESHOLD = 0.01
time_list = [('Start here:', ISS_time[0][1])]
val = 0
for i in range(len(ISS_time)-1):
    # n = type (ISS_time[i][0])
    if ((ISS_time[i+1][0] - ISS_time[i][0]) >= TIME_THRESHOLD):

        if ((val % 2) == 0): #if val is even
            k = ("End here:", ISS_time[i][1])
            
        else:
            k = ("Start here:", ISS_time[i+1][1])
        
        time_list.append(k)
        val = val + 1
# time_list.append(ISS_time[-1])
time_list

[('Start here:', '2022-04-20T04:39:59.616'),
 ('End here:', '2022-04-21T06:15:00.288'),
 ('Start here:', '2022-05-22T10:25:00.192'),
 ('End here:', '2022-05-23T01:34:59.808'),
 ('Start here:', '2022-06-03T22:30:00.000'),
 ('End here:', '2022-06-04T12:09:59.616'),
 ('Start here:', '2022-06-26T01:30:00.000'),
 ('End here:', '2022-06-26T11:05:00.384'),
 ('Start here:', '2022-06-26T13:00:00.288'),
 ('End here:', '2022-06-26T13:14:59.712'),
 ('Start here:', '2022-07-05T19:45:00.288'),
 ('End here:', '2022-07-06T05:35:00.096'),
 ('Start here:', '2022-07-15T03:40:00.192'),
 ('End here:', '2022-07-15T03:40:00.192'),
 ('Start here:', '2022-07-15T14:19:59.808'),
 ('End here:', '2022-07-15T14:30:00.288'),
 ('Start here:', '2022-07-24T03:20:00.096'),
 ('End here:', '2022-07-24T12:09:59.616'),
 ('Start here:', '2022-08-01T17:15:00.000'),
 ('End here:', '2022-08-01T17:19:59.808'),
 ('Start here:', '2022-08-01T18:35:00.384'),
 ('End here:', '2022-08-02T01:59:59.712'),
 ('Start here:', '2022-08-02T03:

In [102]:
# time_list

In [102]:
# Do Fermi next
# FERMI_idx_cut     = np.where(data['Distance(km)FERMI_Light-1'] < THRESHOLD)[0]
# FERMI_time_cut    = data['Time(ModJDate)'][FERMI_idx_cut]
# len(FERMI_time_cut)

1731